In [4]:
CONTRACT_ADDRESS = "0x1e725bcc09ad221d35af5adeda404fb2147b43fa"

# Set to "" to fetch collection name from Opensea
COLLECTION_NAME = ""

# NFTNERDS_BASE_URL = "https://d1h1fvurhxrz4j.cloudfront.net/metadata/"
ALPHASHARKS_BASE_URL = "https://alphasharksv3.s3.us-east-1.amazonaws.com/metadata/"  # 0x1e725bcc09ad221d35af5adeda404fb2147b43fa"

In [5]:
# Get Collection Name from OpenSea
import requests
from honestnft_utils import config


def get_opensea_collection_slug(contract_address: str):

    url = f"https://api.opensea.io/api/v1/asset_contract/{contract_address}"

    headers = {"Accept": "application/json", "X-API-KEY": config.OPENSEA_API_KEY}

    response = requests.request("GET", url, headers=headers)

    return response.json()["collection"]["slug"]

In [ ]:
"""
@date: 14/07/2022
"""

import os
import requests
import time
import datetime
import json
import pandas as pd

from honestnft_utils import config

dt_now = datetime.datetime.utcnow()
start_time = time.time()

print(f"Working in directory: {os.getcwd()}")

# Get Collection Slug
if COLLECTION_NAME == "":
    COLLECTION_NAME = get_opensea_collection_slug(CONTRACT_ADDRESS)

url = f"{ALPHASHARKS_BASE_URL}{CONTRACT_ADDRESS}"

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
}

response = requests.request("GET", url, headers=headers)

if response.status_code == 200:
    response_data = response.json()

    # list to make completeness checks
    token_ids = []

    # TODO
    # - handle collections with no data

    # Create folder to store metadata
    folder = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/"
    print(f"Saving raw attributes to {folder}")
    if not os.path.exists(folder):
        os.mkdir(folder)

    for token in response_data["traitsObj"]:
        token_dict = dict()
        token_ids.append(int(token))
        token_dict["tokenId"] = token
        token_dict["attributes"] = response_data["traitsObj"][token]
        token_dict["image"] = response_data["imagesObj"][token]
        PATH = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/{token}.json"

        with open(PATH, "w") as destination_file:
            json.dump(token_dict, destination_file)

    print(f"tokens in the list {len(token_ids)}")
    print(f"lower_id: {min(token_ids)}")
    print(f"upper_id: {max(token_ids)}")

    if min(token_ids) == 0:
        max_supply = max(token_ids) + 1
        print(f"max supply: {max(token_ids) + 1}")
    else:
        max_supply = max(token_ids)
        print(f"max supply: {max(token_ids)}")

    # warn if nftNerds doesn't have the full collection
    # TODO: print a list of missing token ids
    if len(token_ids) != max_supply:
        print(f"{max_supply - len(token_ids)} tokens with missing metadata")

    # Run pulling.py to grab missing tokens and create rarity csv
    print(
        f"Creating rarity csv with 'pulling.py' using the command: python3 pulling.py --contract {CONTRACT_ADDRESS} --collection {COLLECTION_NAME}"
    )
    os.system(
        f"python3 pulling.py --contract {CONTRACT_ADDRESS} --collection {COLLECTION_NAME}"
    )

    # Calculate rarity_data csv with rarity.py
    print(
        f"Calculating rarity data with 'rarity.py' using the command: python3 rarity.py --collection {COLLECTION_NAME}"
    )
    os.system(f"python3 rarity.py --collection {COLLECTION_NAME}")

    print("--- %s seconds to process collection" % (round(time.time() - start_time, 1)))

elif response.status_code == 403:
    print(
        f"NFT Nerds hasn't published metadata yet. (Error code: {response.status_code})"
    )
else:
    print(f"Received a {response.status_code} error from the NFT Nerds API")

print("finished.")